In [1]:
from typing import List, Union, Tuple, Dict, Optional
import re

Char = str

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms (numbers or strs), possibly ignoring a regex."
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Union[float, int, str]:
    "Parse text into a single float or int or str."
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text

def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]

def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

In [2]:
Message = str   # A string we are trying to match, e.g. "ababba"
Choice  = tuple # A choice of any of the elements, e.g. Choice(([5, 6], [7]))
Pattern = List[Union[Char, int, Choice]]

def parse_messages(rules, messages) -> Tuple[Dict[int, Pattern], List[Message]]:
    "Return a dict of {rule_number: pattern} and a list of messages."
    return dict(map(parse_rule, rules)), messages

def parse_rule(line):
    "Parse '1: 2 3' => (1, [2, 3]); '4: 5, 6 | 7' => (4, Choice(([5, 6], [7])))."
    n, *rhs = atoms(line, ignore='[:"]')
    if '|' in rhs:
        i = rhs.index('|')
        rhs = [Choice((rhs[:i], rhs[i + 1:]))]
    return n, rhs

in19 = parse_messages(*data(19, str.splitlines, sep='\n\n'))

In [12]:
def match(pat, msg, rules) -> Optional[Message]:
    "If a prefix of msg matches pat, return remaining str; else None"
    print(pat, msg)
    print(pat and not msg, not pat)
    if pat and not msg:              # Failed to match whole pat
        return None
    elif not pat:                    # Matched whole pat
        return msg
    elif pat[0] == msg[0]:           # Matched first char; continue
        return match(pat[1:], msg[1:], rules)
    elif isinstance(pat[0], int):    # Look up the rule number
        return match(rules[pat[0]] + pat[1:], msg, rules)
    elif isinstance(pat[0], Choice): # Match any of the choices
        for choice in pat[0]:
            m = match(choice + pat[1:], msg, rules)
            if m is not None:
                return m
    return None

def day19_1(inputs):
    "How many messages completely match rule 0?"
    rules, messages = inputs
#     print(messages[0])
    return quantify(match(rules[0], msg, rules) == ''
                    for msg in messages[:1])

day19_1(in19)

[8, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[42, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[([119, 58], [111, 57]), 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[119, 58, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[([58, 71], [57, 33]), 58, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[58, 71, 58, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
['b', 71, 58, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[57, 33, 58, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
['a', 33, 58, 11] aaaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[33, 58, 11] aaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[([58, 101], [57, 26]), 58, 11] aaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[58, 101, 58, 11] aaabbaaaaababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
['b', 101, 58, 11] aaabbaaaaababbaabbbbbbbabbbb

['b', 99, 58, 57, 31] aababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[57, 75, 58, 57, 31] aababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
['a', 75, 58, 57, 31] aababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[75, 58, 57, 31] ababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[([58, 1], [57, 67]), 58, 57, 31] ababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[58, 1, 58, 57, 31] ababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
['b', 1, 58, 57, 31] ababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[57, 67, 58, 57, 31] ababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
['a', 67, 58, 57, 31] ababbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[67, 58, 57, 31] babbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[([94, 57], [125, 58]), 58, 57, 31] babbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[94, 57, 58, 57, 31] babbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[([57, 57], [58, 57]), 57, 58, 57, 31] babbaabbbbbbbabbbbaabbabaabbbbbbbaaba
False False
[57, 57, 57, 58, 57, 31] 

0

In [6]:
def day19_2(inputs):
    "How many messages completely match rule 0, with new rules 8 and 11?"
    rules, messages = inputs
    rules2 = {**rules, 8: [42, maybe(8)], 11: [42, maybe(11), 31]}
    return day19_1((rules2, messages))
             
def maybe(n): return Choice(([], [n]))

day19_2(in19)

321